# Finding Similar Items: Textually Similar Documents

In [93]:
import findspark
findspark.init()

from pyspark import *
from pyspark.sql.functions import desc, col
from pyspark.sql import *
from graphframes import *

import os
from IPython.display import display, HTML
import pandas as pd
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# https://graphframes.github.io/graphframes/docs/_site/quick-start.html
# https://stackoverflow.com/questions/65011599/how-to-start-graphframes-on-spark-on-pyspark-on-juypter-on-docker
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages graphframes:graphframes:0.8.1-spark3.0-s_2.12 pyspark-shell'

In [4]:
spark = SparkSession.builder.appName('hw1').getOrCreate()

21/11/07 17:15:57 WARN Utils: Your hostname, mark-machine resolves to a loopback address: 127.0.1.1; using 10.3.49.83 instead (on interface wlp8s0)
21/11/07 17:15:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/mark/.ivy2/cache
The jars for the packages stored in: /home/mark/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-cf8afc35-1786-4bc4-bb3e-ed5203109f4d;1.0
	confs: [default]
	found graphframes#graphframes;0.8.1-spark3.0-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 172ms :: artifacts dl 6ms
	:: modules in use:
	graphframes#graphframes;0.8.1-spark3.0-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-------------------------------------

http://mlg.ucd.ie/datasets/bbc.html

In [8]:
%%bash
file=./data/bbc-fulltext.zip

if [ -e "$file" ]; then
    echo "$file exists, skipping"
else 
    echo "$file does not exist, using wget to download it"
    wget http://mlg.ucd.ie/files/datasets/bbc-fulltext.zip -P data/
    unzip $file -d data/
fi

./data/bbc-fulltext.zip does not exist, using wget to download it
Archive:  ./data/bbc-fulltext.zip
   creating: data/bbc/
   creating: data/bbc/business/
  inflating: data/bbc/business/001.txt  
  inflating: data/bbc/business/002.txt  
  inflating: data/bbc/business/003.txt  
  inflating: data/bbc/business/004.txt  
  inflating: data/bbc/business/005.txt  
  inflating: data/bbc/business/006.txt  
  inflating: data/bbc/business/007.txt  
  inflating: data/bbc/business/008.txt  
  inflating: data/bbc/business/009.txt  
  inflating: data/bbc/business/010.txt  
  inflating: data/bbc/business/011.txt  
  inflating: data/bbc/business/012.txt  
  inflating: data/bbc/business/013.txt  
  inflating: data/bbc/business/014.txt  
  inflating: data/bbc/business/015.txt  
  inflating: data/bbc/business/016.txt  
  inflating: data/bbc/business/017.txt  
  inflating: data/bbc/business/018.txt  
  inflating: data/bbc/business/019.txt  
  inflating: data/bbc/business/020.txt  
  inflating: data/bbc/bus

--2021-11-07 17:20:16--  http://mlg.ucd.ie/files/datasets/bbc-fulltext.zip
Resolving mlg.ucd.ie (mlg.ucd.ie)... 137.43.93.132
Connecting to mlg.ucd.ie (mlg.ucd.ie)|137.43.93.132|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2874078 (2,7M) [application/zip]
Saving to: ‘data/bbc-fulltext.zip’

     0K .......... .......... .......... .......... ..........  1% 37,8K 73s
    50K .......... .......... .......... .......... ..........  3% 44,3K 66s
   100K .......... .......... .......... .......... ..........  5%  112M 43s
   150K .......... .......... .......... .......... ..........  7% 80,8K 40s
   200K .......... .......... .......... .......... ..........  8% 8,19M 31s
   250K .......... .......... .......... .......... .......... 10% 82,3K 31s
   300K .......... .......... .......... .......... .......... 12% 81,4K 30s
   350K .......... .......... .......... .......... .......... 14% 10,4M 26s
   400K .......... .......... .......... .......... .......... 1

In [81]:
def read_text(path):
    print(path)
    text = np.nan
    try:
        with open(path, 'r') as f:
            lines = f.readlines()

        lines = [line.strip() for line in lines]
        lines = list(filter(lambda x: x != "", lines))

        text = " ".join(lines)
    except Exception as e:
        print(f"skipping {path} due to: {e}")
    finally:
        return text

path = "data/bbc/sport/199.txt"
read_text(path)

data/bbc/sport/199.txt
skipping data/bbc/sport/199.txt due to: 'utf-8' codec can't decode byte 0xa3 in position 257: invalid start byte


nan

In [82]:
def read_text2(path):
    text = None
    try:
        text = pd.read_csv(path, encoding= 'unicode_escape')
    except Exception as e:
        print(f"skipping file at: {path} due to: {e}")
    else:
        return text

path = "data/bbc/business/001.txt"
read_text(path)

data/bbc/business/001.txt


'Ad sales boost Time Warner profit Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL. Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL\'s underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL\

In [86]:
data_path = "data/bbc/"
topic_names = ["business", "entertainment", "politics", "sport", "tech"]

data = {}
paths = []
topics = []
texts = []

for topic_name in topic_names:
    dir_path = os.path.join(data_path, topic_name)
    
    for dirpath, dirnames, filenames in os.walk(dir_path):
        paths_ = [os.path.join(dirpath, filename) for filename in filenames]
        topics_ = [topic_name for i in range(len(paths_))]
        
        texts_ = [read_text(path_) for path_ in paths_]
        
        paths += paths_
        topics += topics_
        texts += texts_

data["path"] = paths
data["topic"] = topics
data["text"] = texts

df = pd.DataFrame(data=data)
df

data/bbc/business/489.txt
data/bbc/business/194.txt
data/bbc/business/461.txt
data/bbc/business/007.txt
data/bbc/business/145.txt
data/bbc/business/463.txt
data/bbc/business/294.txt
data/bbc/business/505.txt
data/bbc/business/348.txt
data/bbc/business/159.txt
data/bbc/business/123.txt
data/bbc/business/077.txt
data/bbc/business/002.txt
data/bbc/business/509.txt
data/bbc/business/370.txt
data/bbc/business/050.txt
data/bbc/business/072.txt
data/bbc/business/011.txt
data/bbc/business/314.txt
data/bbc/business/052.txt
data/bbc/business/355.txt
data/bbc/business/155.txt
data/bbc/business/412.txt
data/bbc/business/491.txt
data/bbc/business/105.txt
data/bbc/business/497.txt
data/bbc/business/061.txt
data/bbc/business/404.txt
data/bbc/business/125.txt
data/bbc/business/070.txt
data/bbc/business/477.txt
data/bbc/business/349.txt
data/bbc/business/237.txt
data/bbc/business/103.txt
data/bbc/business/167.txt
data/bbc/business/217.txt
data/bbc/business/211.txt
data/bbc/business/043.txt
data/bbc/bus

data/bbc/sport/412.txt
data/bbc/sport/491.txt
data/bbc/sport/105.txt
data/bbc/sport/497.txt
data/bbc/sport/061.txt
data/bbc/sport/404.txt
data/bbc/sport/125.txt
data/bbc/sport/070.txt
data/bbc/sport/477.txt
data/bbc/sport/349.txt
data/bbc/sport/237.txt
data/bbc/sport/103.txt
data/bbc/sport/167.txt
data/bbc/sport/217.txt
data/bbc/sport/211.txt
data/bbc/sport/043.txt
data/bbc/sport/107.txt
data/bbc/sport/342.txt
data/bbc/sport/284.txt
data/bbc/sport/005.txt
data/bbc/sport/494.txt
data/bbc/sport/222.txt
data/bbc/sport/335.txt
data/bbc/sport/433.txt
data/bbc/sport/081.txt
data/bbc/sport/035.txt
data/bbc/sport/060.txt
data/bbc/sport/467.txt
data/bbc/sport/185.txt
data/bbc/sport/366.txt
data/bbc/sport/028.txt
data/bbc/sport/212.txt
data/bbc/sport/025.txt
data/bbc/sport/401.txt
data/bbc/sport/276.txt
data/bbc/sport/300.txt
data/bbc/sport/420.txt
data/bbc/sport/003.txt
data/bbc/sport/280.txt
data/bbc/sport/449.txt
data/bbc/sport/235.txt
data/bbc/sport/055.txt
data/bbc/sport/213.txt
data/bbc/sp

,path,topic,text
0,data/bbc/business/489.txt,business,Criminal probe on Citigroup deals Traders at U...
1,data/bbc/business/194.txt,business,'Post-Christmas lull' in lending UK mortgage l...
2,data/bbc/business/461.txt,business,Beijingers fume over parking fees Choking traf...
3,data/bbc/business/007.txt,business,Jobs growth still slow in the US The US create...
4,data/bbc/business/145.txt,business,Winn-Dixie files for bankruptcy US supermarket...
...,...,...,...
2220,data/bbc/tech/232.txt,tech,BBC web search aids odd queries The BBC's onli...
2221,data/bbc/tech/117.txt,tech,Joke e-mail virus tricks users A virus that di...
2222,data/bbc/tech/177.txt,tech,Microsoft debuts security tools Microsoft is r...
2223,data/bbc/tech/394.txt,tech,TV's future down the phone line Internet TV ha...


In [89]:
df.dropna(inplace=True)

In [91]:
data = spark.createDataFrame(df, list(df.columns.values))

DataFrame[path: string, topic: string, text: string]

In [92]:
data.show()

+--------------------+--------+--------------------+
|                path|   topic|                text|
+--------------------+--------+--------------------+
|data/bbc/business...|business|Criminal probe on...|
|data/bbc/business...|business|'Post-Christmas l...|
|data/bbc/business...|business|Beijingers fume o...|
|data/bbc/business...|business|Jobs growth still...|
|data/bbc/business...|business|Winn-Dixie files ...|
|data/bbc/business...|business|US economy still ...|
|data/bbc/business...|business|Wall Street cheer...|
|data/bbc/business...|business|China now top tra...|
|data/bbc/business...|business|US to probe airli...|
|data/bbc/business...|business|India unveils ant...|
|data/bbc/business...|business|Train strike grip...|
|data/bbc/business...|business|China had role in...|
|data/bbc/business...|business|Dollar gains on G...|
|data/bbc/business...|business|Euro firms miss o...|
|data/bbc/business...|business|US manufacturing ...|
|data/bbc/business...|business|Bad weather hit

In [ ]:
class Shingling():
    def __init__(self, k, df):
        # build k shingle map from corpus of docs
        pass
    
    def __call__(self, row):
        # get text from row and convert it to k-shingles
        pass
    
class CompareSets():
    def __init__(self,):
        pass
    
    def __call__(self, row):
        # compute Jaccard sim between two sets of shingles
        pass
    